In [4]:
pip install pyscipopt

Note: you may need to restart the kernel to use updated packages.


In [101]:
from pyscipopt import Model, quicksum, multidict
import numpy as np
import pandas as pd
import geopandas as gpd
import random

In [103]:
# adapted from https://scipbook.readthedocs.io/en/latest/flp.html
def flp(I,J,d,M,c,existing_sites=None):
    model = Model("flp")
    x,y = {},{}
    for j in J:
        y[j] = model.addVar(vtype="B", name="y(%s)"%j)
        for i in I:
            x[i,j] = model.addVar(vtype="C", name="x(%s,%s)"%(i,j))
    for i in I:
        model.addCons(quicksum(x[i,j] for j in J) == d[i], "Demand(%s)"%i)
    for j in M:
        model.addCons(quicksum(x[i,j] for i in I) <= M[j]*y[j], "Capacity(%s)"%i)
    for (i,j) in x:
        model.addCons(x[i,j] <= d[i]*y[j], "Strong(%s,%s)"%(i,j))
    
    if existing_sites:
        for j in existing_sites:
            model.addCons(y[j] == 1, name=f"ForceOpen({j})")

    model.addCons(quicksum(y[j] for j in J) <= 6, "FacilityLimit") 
            
    model.setObjective(
        quicksum(c[i,j]*x[i,j] for i in I for j in J),
        "minimize")
    model.data = x,y
    return model

In [105]:
# for I, d make a dictionary of planning units to number of students
pu_data = gpd.read_file('GIS_files/pu_with_proj_SPLIT.geojson').set_index('pu_2324_84')
pu_data = pu_data['final_proj'].to_dict()

I, d = multidict(pu_data)

In [117]:
# for J, M make a dictionary of sites to capacities
schools = gpd.read_file('/Users/leahwallihan/Durham_school_planning/DPS-Planning/GIS_files/dps_hs_locations.geojson')
schools = schools.to_crs('EPSG:4326')
pu = gpd.read_file('/Users/leahwallihan/Durham_school_planning/geospatial files/pu_shape_new.geojson')
pu = pu.to_crs('EPSG:4326')

for i, geometry in enumerate(pu['geometry']):
    in_geometry = geometry.contains(schools['geometry'])
    pu_id = pu.loc[i, 'OBJECTID']

    schools.loc[in_geometry, 'pu'] = pu_id

In [119]:
# let's remove planning units in the North from J to make problem simpler
not_north = pu[(pu['Region'] != 'Central')]

# initialize dictionary of planning units with capacity of 1600 for potential site
pu_dict = {}
for _, row in not_north.iterrows():
    pu_dict[row['OBJECTID']] = 1600

# find which planning units have existing school
schools['pu'] = None

for i, geometry in enumerate(pu['geometry']):
    in_geometry = schools.within(geometry)
    pu_id = pu.loc[i, 'OBJECTID']
    schools.loc[in_geometry, 'pu'] = pu_id

# replace capacities of planning units with existing schools
pu_dict[45] = 1600
pu_dict[507] = 1810
pu_dict[602] = 1540
pu_dict[566] = 1540
pu_dict[290] = 1535

J, M = multidict(pu_dict)

# define which sites already exist
existing_sites = {602, 290, 45, 566, 507}

In [121]:
# create distance matrix
c = {}

pu_centroids = pu.set_index('OBJECTID').geometry.centroid 

for i in I:
    for j in J:
        dist = pu_centroids[i].distance(pu_centroids[j])
        c[i, j] = dist

/var/folders/ml/wj07z83j0yq4rjs8n_7sv0mw0000gn/T/ipykernel_5998/1973786647.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pu_centroids = pu.set_index('OBJECTID').geometry.centroid


In [18]:
# '''
# for testing:
I_small = random.sample(I, 100)
d_small = {i: d[i] for i in I_small}
c_small = {(i,j): c[i,j] for i in I_small for j in J if (i,j) in c}

model = flp(I_small, J, d_small, M, c_small, existing_sites=existing_sites)
model.setParam('limits/solutions', 3)
model.optimize()
EPS = 1.e-6
x,y = model.data
edges = [(i,j) for (i,j) in x if model.getVal(x[i,j]) > EPS]
facilities = [j for j in y if model.getVal(y[j]) > EPS]
print ("Optimal value=", model.getObjVal())
print ("Facilities at nodes:", facilities)
print ("Edges:", edges)
# '''

presolving:
(round 1, fast)       18355 del vars, 18380 del conss, 0 add conss, 73405 chg bounds, 0 chg sides, 734 chg coeffs, 0 upgd conss, 0 impls, 1 clqs
(round 2, fast)       18355 del vars, 18760 del conss, 0 add conss, 73405 chg bounds, 0 chg sides, 734 chg coeffs, 0 upgd conss, 0 impls, 1 clqs
(round 3, fast)       18430 del vars, 18760 del conss, 0 add conss, 73405 chg bounds, 0 chg sides, 734 chg coeffs, 0 upgd conss, 0 impls, 1 clqs
(round 4, fast)       18430 del vars, 18760 del conss, 0 add conss, 73405 chg bounds, 75 chg sides, 734 chg coeffs, 0 upgd conss, 0 impls, 1 clqs
(round 5, fast)       51023 del vars, 51186 del conss, 0 add conss, 73405 chg bounds, 75 chg sides, 1463 chg coeffs, 0 upgd conss, 0 impls, 1 clqs
(round 6, exhaustive) 51023 del vars, 51186 del conss, 0 add conss, 73405 chg bounds, 75 chg sides, 1463 chg coeffs, 22250 upgd conss, 0 impls, 1 clqs
(round 7, exhaustive) 51112 del vars, 51186 del conss, 0 add conss, 73405 chg bounds, 75 chg sides, 1463 chg 

In [20]:
solution_reports = []

# Get all stored solutions
sols = model.getSols()

for sidx, sol in enumerate(sols):
    assignments = {}

    for (i_, j_) in x:
        if model.getSolVal(sol, x[i_, j_]) > 0.5:
            if j_ not in assignments:
                assignments[j_] = []
            assignments[j_].append(i_)

    student_count = {}
    if 'students' in globals():  
        for j_, pus in assignments.items():
            student_count[j_] = sum(students.get(i_, 0) for i_ in pus)

    solution_reports.append({
        'solution_number': sidx + 1,
        'facilities': list(assignments.keys()),
        'assignments': assignments,
        'student_count': student_count if 'students' in globals() else None
    })

In [22]:
for report in solution_reports:
    print(f"\n--- Solution #{report['solution_number']} ---")
    print("Facilities opened:", report['facilities'])

    print("Assignments:")
    for fac, pus in report['assignments'].items():
        print(f"  Facility {fac} <-- Planning Units {pus}")

    if report['student_count']:
        print("Student Count per Facility:")
        for fac, count in report['student_count'].items():
            print(f"  Facility {fac}: {count} students")


--- Solution #1 ---
Facilities opened: [45, 208, 290, 507, 566, 602]
Assignments:
  Facility 45 <-- Planning Units [435, 156, 199, 451]
  Facility 208 <-- Planning Units [443, 582, 766, 572, 371, 548, 141, 368, 549, 442, 495]
  Facility 290 <-- Planning Units [290, 735, 380, 547, 535, 120, 275, 391, 399, 654, 530, 423, 450, 623, 424, 417, 842, 295, 738]
  Facility 507 <-- Planning Units [112, 497, 791, 266, 650, 509, 277, 49, 532, 4, 803]
  Facility 566 <-- Planning Units [722, 202, 661, 668, 203, 188, 333, 190, 671, 364]
  Facility 602 <-- Planning Units [158, 602, 750, 726, 322, 678, 755, 758, 96, 91, 305, 808, 580, 616, 70, 718, 806, 810, 687, 704]

--- Solution #2 ---
Facilities opened: [1, 45, 290, 507, 566, 602]
Assignments:
  Facility 1 <-- Planning Units [722, 188, 333, 190, 156]
  Facility 45 <-- Planning Units [443, 435, 582, 766, 572, 371, 199, 141, 368, 549, 442, 495, 451]
  Facility 290 <-- Planning Units [290, 735, 380, 547, 535, 120, 275, 391, 399, 654, 530, 423, 548, 4

In [ ]:
model = flp(I, J, d, M, c, existing_sites=existing_sites)
model.setParam('limits/solutions', 7)
model.optimize()
EPS = 1.e-6
x,y = model.data
edges = [(i,j) for (i,j) in x if model.getVal(x[i,j]) > EPS]
facilities = [j for j in y if model.getVal(y[j]) > EPS]
print ("Optimal value=", model.getObjVal())
print ("Facilities at nodes:", facilities)
print ("Edges:", edges)

In [45]:
sol = model.getBestSol()

assignments = {}
for (i_, j_) in x:
    if model.getSolVal(sol, x[i_, j_]) > 0.5:
        if j_ not in assignments:
            assignments[j_] = []
        assignments[j_].append(i_)


In [57]:
pu_to_facility = {}
for facility, pu_list in assignments.items():
    for pu_id in pu_list:
        pu_to_facility[pu_id] = facility

pu['assignment'] = pu['OBJECTID'].map(pu_to_facility)


In [61]:
pu.to_file('CFLP_solution.geojson', driver='GeoJSON')

In [16]:
schools

,OBJECTID,facilityid,sch_id6,sch_id,name,address,phone,agencyurl,operhours,spectype,...,region_physical,calendar_2324,calendar_2425,choice_2324,choice_2425,choice_2526,x_ncplane,y_ncplane,geometry,pu
0,11,27253,320368,368,Southern High School,"800 Clayton Road Durham, NC 27703",919-560-3968,http://southern.dpsnc.net/pages/Southern_High,9:15am - 4:15pm,Magnet School,...,east,Traditional,Traditional,Energy & Sustainability,Energy & Sustainability,Energy & Sustainability,2050552.39,818903.23,POINT (-78.8291 35.9999),602
1,15,87951,320325,325,Hillside High School,3727 Fayetteville Street Durham NC 27707,919-560-3925,http://hillside.dpsnc.net/pages/Hillside_High,9:15am - 4:15pm,None,...,southeast,Traditional,Traditional,International Baccalaureate,International Baccalaureate,International Baccalaureate,2028531.50,801909.91,POINT (-78.9036 35.9533),290
2,56,6494,320356,356,Northern High School,4622 N Roxboro Road Durham NC 27712,919-560-3956,http://northern.dpsnc.net/pages/Northern_High,9:15am - 4:15pm,None,...,north,Traditional,Traditional,None,None,None,2029463.13,841220.63,POINT (-78.90032 36.06129),45
3,57,12065,320365,365,Riverside High School,"3218 Rose of Sharon Road Durham, NC 27712",919-560-3965,http://www.edlinesites.net/pages/Riverside_High,9:15am - 4:15pm,None,...,north,Traditional,Traditional,None,None,None,2018531.26,843582.30,POINT (-78.9373 36.0678),566
4,58,76194,320312,312,Jordan High School,6806 Garrett Road Durham NC 27707,919-560-3912,http://jordan.dpsnc.net/pages/Jordan_High,9:15am - 4:15pm,None,...,southwest,Traditional,Traditional,None,None,None,2011193.93,790759.00,POINT (-78.9622 35.9227),507


In [26]:
new_pu = gpd.read_file('/Users/leahwallihan/Durham_school_planning/DPS-Planning/GIS_files/pu_2324_SPLIT.geojson', driver='GeoJSON')

/opt/anaconda3/envs/spatialdata/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(


In [28]:
new_pu

,OBJECTID,pu_2324_84,X,Y,M_min,PS_ID,PUID2122_2,ps_id_833,psid_982,TIMS_PU,Region,Shape_Length,Shape_Area,geometry
0,1,1,2.049028e+06,893128.606346,0.0,589,589,1,1,PU589,North,26532.120021,3.556198e+07,"POLYGON ((-8774487.947 4327319.083, -8774521.6..."
1,2,2,2.039923e+06,835434.121317,0.0,401,401,2,2,PU401,North,19333.085511,2.080136e+07,"POLYGON ((-8778905.777 4307747.962, -8778594.2..."
2,3,3,2.058005e+06,797649.028742,0.0,782,782.0,3,3,PU782.0,East,18314.823119,1.329077e+07,"POLYGON ((-8771275.633 4292943.901, -8771301.3..."
3,4,4,2.003222e+06,772895.737307,0.0,201,201.0,4,4,PU201.0,Southwest,29282.100083,4.684140e+07,"POLYGON ((-8792747.171 4285053.725, -8792710.5..."
4,5,5,2.067334e+06,828244.230730,0.0,646,646,6,6,PU646,East,60685.151474,1.497046e+08,"POLYGON ((-8766810.66 4304937.339, -8765684.83..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846,844,844,2.011851e+06,824885.632280,0.0,468,468,290,290,PU468,Southwest,6419.265826,2.254156e+06,"POLYGON ((-8789535.54 4303186.77, -8789542.711..."
847,845,845,2.018359e+06,853286.849340,0.0,329,329,107,107,PU329,North,15620.191583,1.246839e+07,"POLYGON ((-8786600.606 4314310.08, -8786600.86..."
848,846,846,2.016360e+06,858711.587472,0.0,573,573,77,883,PU573,North,16660.241935,9.218197e+06,"POLYGON ((-8787335.076 4316169.87, -8787334.22..."
849,847,847,2.013328e+06,854621.040074,0.0,322,322,50,50,PU322,North,9379.857191,3.748151e+06,"POLYGON ((-8788811.468 4314719.369, -8788811.4..."
